In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
from matplotlib import animation, rc
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from IPython.display import HTML

In [ ]:
from optibot.schemes import interpolated_array,trap_interp, trapz_step, integrate_trapz

In [ ]:
from optibot.numpy import trapz_integrate, num_derivative

In [ ]:
from optibot.schemes import is2d
def F_ej(x,u,params):
    if is2d(x):
        f = np.zeros_like(x)
        f[:,0] = x[:,2]
        f[:,1] = x[:,3]
        f[:,2] = u[:,0]
        f[:,3] = u[:,1]
    else:
        f = np.zeros_like(x)
        f[0] = x[2]
        f[1] = x[3]
        f[2] = u[0]
        f[3] = u[1]
    return f

In [ ]:
_x_0 = np.array([0.,0.,0.,0.])
_ej_u = np.array([[1.,1.],[1.,1.]])
_x_1 = trapz_step(_x_0, _ej_u[0], _ej_u[1], F_ej, 1., [0.,])

In [ ]:
_x_1

In [ ]:
_ej_x = np.array([_x_0, _x_1])

_ej_x

In [ ]:
_X_interp, _U_interp = interpolated_array(_ej_x, _ej_u, F_ej, 1.,
                                  np.array([0,0.5,1]), [1], 'trapz')
_X_interp

In [ ]:
_ej_u_2 = np.sin(np.linspace([0,0], [20,20], 51))
_ej_x_2 = np.array(integrate_trapz(
    _x_0,
    _ej_u_2,
    F_ej,
    0.1,
    [1,]
))[:-1]
_ej_x_2.shape

In [ ]:
_ej_x_interp, _ej_u_interp = interpolated_array(_ej_x_2, _ej_u_2, F_ej, 0.1,
                                  np.linspace(0, 5,101), [1,], 'trapz')

In [ ]:
plt.figure(figsize=[50,20])
aa = 2
ej_t_array = np.linspace(0, 5,51)
new_ej_t_array = np.linspace(0, 5,101)
plt.plot(new_ej_t_array, _ej_x_interp[:,aa], 'b')
plt.plot(ej_t_array, _ej_x_2[:,aa], 'r:')
plt.plot(ej_t_array, _ej_x_2[:,aa], 'g:')
plt.plot(ej_t_array, _ej_x_2[:,aa], 'ko')
plt.vlines(new_ej_t_array, -0.1, 0.6)

In [ ]:
h = 0.1

In [ ]:
accelerations = F_ej(_ej_x_2, _ej_u_2,[1,])
accelerations

In [ ]:
_err_1 = accelerations[:,2] - num_derivative(accelerations[:,0], h)
_err_2 = accelerations[:,0] - _ej_x_2[:,2]
_err_3 = _ej_x_2[:,2] - num_derivative(_ej_x_2[:,0],h)

In [ ]:
plt.figure(figsize=[15,7])
plt.plot(ej_t_array,_err_1, label = f'error 1')
plt.plot(ej_t_array,_err_2, label = f'error 2')
plt.plot(ej_t_array,_err_3, label = f'error 3')
plt.plot([0, ej_t_array[-1]], [0,0], 'k', lw=1, ls = ':')
plt.legend()

In [ ]:
_err_4 = trapz_integrate(accelerations[:,2], ej_t_array) - accelerations[:,0]
_err_5 = trapz_integrate(_ej_x_2[:,2], ej_t_array) - _ej_x_2[:,0]

In [ ]:
plt.figure(figsize=[15,7])
plt.plot(ej_t_array,_err_4, label = f'error 4')
plt.plot(ej_t_array,_err_5, label = f'error 5')
plt.plot([0, ej_t_array[-1]], [0,0], 'k', lw=1, ls = ':')
plt.legend()